In [1]:
import pandas as pd
import requests
import json

In [2]:
dat = pd.read_csv("../data/pnwNP_StatsLocations.csv")

In [3]:
i=2

In [4]:
y_loc = dat["dec_lat_va"][i]
x_loc = dat["dec_long_va"][i]
state = 'OR'
gage = dat["gage"][i]
gage

10378500

In [10]:
# Service URLS
StreamStatsServiceURLS = {
    'watershed': 'https://streamstats.usgs.gov/streamstatsservices/watershed.geojson',
    'basinCharacteristics': 'https://streamstats.usgs.gov/streamstatsservices/parameters.json',
    'gage': 'https://streamstats.usgs.gov/gagestatsservices/stations'
    }
NSSServiceURlS = {
    'regressionRegions': 'https://streamstats.usgs.gov/nssservices/regressionregions/bylocation',
    'scenarios': 'https://streamstats.usgs.gov/nssservices/scenarios',
    'computeFlowStats': 'https://streamstats.usgs.gov/nssservices/scenarios/Estimate'
}

# Declare cookies for use of consistent StreamStats servers
global cookies 

In [6]:
# Step 1 - Get the watershed
print('Step 1 - Get the watershed')
watershedURLParams = {
    'rcode': state, 
    'xlocation': x_loc, 
    'ylocation': y_loc,
    'crs': 4326,
#     'crs': 4269,
}
delineatedBasin = None
while delineatedBasin is None: # Sometimes the watershed endpoint returns with no geometry - this loop is written to keep trying until a geometry is returned
    print('Delineating basin...')
    basinDelineationResponse = requests.get(url = StreamStatsServiceURLS['watershed'], params = watershedURLParams)
    if basinDelineationResponse.status_code == 200:
        cookies = basinDelineationResponse.cookies
        try:
            delineatedBasin = json.loads(basinDelineationResponse.content.decode('utf-8'))["featurecollection"][1]["feature"]["features"][0]["geometry"] # this will be used in step 2
            workspaceID = json.loads(basinDelineationResponse.content.decode('utf-8'))["workspaceID"] # this will be used in step 4
        except:
            print("No geometry returned. Retrying...")
    else:
        print("Error. Retrying...")

Step 1 - Get the watershed
Delineating basin...


In [7]:
# Step 2 - Get the Regression Regions associated with the watershed
print('Step 2 - Get the Regression Regions associated with the watershed')
regressionRegionPOSTBody = delineatedBasin # from step 1

regressionRegions = None
while regressionRegions == None:
    regressionRegionsResponse = requests.post(url = NSSServiceURlS['regressionRegions'], json=regressionRegionPOSTBody)
    if regressionRegionsResponse.status_code == 200:
        regressionRegions = json.loads(regressionRegionsResponse.content.decode('utf-8'))
        regressionRegionCodes = [ sub['code'] for sub in regressionRegions ] # this will be used in step 3
    else:
        print("Error.")

Step 2 - Get the Regression Regions associated with the watershed


In [8]:
# Step 3 - Get the Scenarios associated with the Regression Regions
print('Step 3 - Get the Scenarios associated with the Regression Regions')

scenarioURLParams = {
    'regions': state, 
    'statisticgroups': 4, # this can be updated depending on which flow statistics you want to calculate (2 is associated with Peak Flow Statistics) - you can find all statistic groups available for your region at https://streamstats.usgs.gov/docs/nssservices/#/StatisticGroups/GET/StatisticGroups
    'regressionregions': (','.join(regressionRegionCodes)) # from step 2, needs to be comma separated list
}
scenarioResponse = requests.get(url = NSSServiceURlS['scenarios'], params=scenarioURLParams)

parameters = None
while parameters == None:
    if scenarioResponse.status_code == 200:
        scenarios = json.loads(scenarioResponse.content.decode('utf-8'))[0] # this will be used in step 5
        parameters = json.loads(scenarioResponse.content.decode('utf-8'))[0]["regressionRegions"][0]["parameters"]
        parameterCodes = [ sub['code'] for sub in parameters ] # this will be used in step 4
    else:
        print("Error.")


Step 3 - Get the Scenarios associated with the Regression Regions


In [11]:
# Step 4 - Compute the basin characteristics
print('Step 4 - Compute the basin characteristics')
basinCharacteristicsURLParams = {
    'rcode': state, 
    'workspaceID': workspaceID, # from step 1
    'includeparameters': (','.join(parameterCodes)) # from step 3
}

basinCharacteristics = None
while basinCharacteristics is None:
    basinCharacteristicsResponse = requests.get(url = StreamStatsServiceURLS['basinCharacteristics'], params=basinCharacteristicsURLParams, cookies=cookies)
    if basinCharacteristicsResponse.status_code == 200:
        basinCharacteristics = json.loads(basinCharacteristicsResponse.content.decode('utf-8')) # this will be used in step 5
    else:
        print("Error.")

Step 4 - Compute the basin characteristics


In [12]:
# Step 5 - Compute Flow Statistics
print('Step 5 - Compute Flow Statistics')
flowStatsURLParams = {
    'regions': state
}
flowStatsPOSTBody = [] 
# for counter, x in enumerate(scenarios['regressionRegions'][0]['parameters']): # from step 3 & step 4
#     for p in basinCharacteristics['parameters']:
#         if x['code'].lower() == p['code'].lower():
#             scenarios['regressionRegions'][0]['parameters'][counter]['value'] = p['value']
flowStatsPOSTBody.append(scenarios)

flowStats = None
while flowStats == None:
    flowStatsResponse = requests.post(url = NSSServiceURlS['computeFlowStats'], params = flowStatsURLParams, json=flowStatsPOSTBody)
    if flowStatsResponse.status_code == 200:
        flowStats = json.loads(flowStatsResponse.content.decode('utf-8'))
        print(flowStats)
    else:
        print("Error.")

Step 5 - Compute Flow Statistics
[{'statisticGroupID': 4, 'statisticGroupName': 'Low-Flow Statistics', 'regressionRegions': [{'id': 489, 'name': 'LowFlow_Ann_Region08_2008_5126', 'code': 'GC1296', 'description': '', 'statusID': 4, 'citationID': 119, 'parameters': [{'id': 11599, 'name': 'Drainage Area', 'description': 'Area that drains to a point on a stream', 'code': 'DRNAREA', 'unitType': {'id': 35, 'unit': 'square miles', 'abbr': 'mi^2'}, 'value': -999.99, 'limits': {'max': 1591.119, 'min': 18.324}}, {'id': 11600, 'name': 'Mean Annual Precipitation', 'description': 'Mean Annual Precipitation', 'code': 'PRECIP', 'unitType': {'id': 36, 'unit': 'inches', 'abbr': 'in'}, 'value': -999.99, 'limits': {'max': 80.1552, 'min': 13.8701}}], 'results': [{'id': 0, 'name': '7 Day 2 Year Low Flow', 'code': 'M7D2Y', 'description': '7-Day mean low-flow that occurs on average once in 2 years', 'value': -99999.0, 'unit': {'id': 44, 'unit': 'cubic feet per second', 'abbr': 'ft^3/s'}, 'equation': 'max(0,1

In [15]:
flowStatsResponse.url

'https://streamstats.usgs.gov/nssservices/scenarios/Estimate?regions=OR'

In [16]:
flowStatsURLParams

{'regions': 'OR'}

In [17]:
flowStatsPOSTBody

[{'statisticGroupID': 4,
  'statisticGroupName': 'Low-Flow Statistics',
  'regressionRegions': [{'id': 489,
    'name': 'LowFlow_Ann_Region08_2008_5126',
    'code': 'GC1296',
    'description': '',
    'statusID': 4,
    'citationID': 119,
    'parameters': [{'id': 11599,
      'name': 'Drainage Area',
      'description': 'Area that drains to a point on a stream',
      'code': 'DRNAREA',
      'unitType': {'id': 35, 'unit': 'square miles', 'abbr': 'mi^2'},
      'value': -999.99,
      'limits': {'max': 1591.119, 'min': 18.324}},
     {'id': 11600,
      'name': 'Mean Annual Precipitation',
      'description': 'Mean Annual Precipitation',
      'code': 'PRECIP',
      'unitType': {'id': 36, 'unit': 'inches', 'abbr': 'in'},
      'value': -999.99,
      'limits': {'max': 80.1552, 'min': 13.8701}}]}],
  'links': [{'rel': 'Citations',
    'href': 'https://streamstats.usgs.gov/nssservices/citations?regressionregions=489',
    'method': 'GET'}]}]